In [1]:
import os
from datetime import datetime

# Set the settings module
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "server.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
# Set up Django
django.setup()

from mims.models import MIMSImage
from mims.services import create_alignment_estimates


In [38]:

import numpy as np
import pyvips
import os
from datetime import datetime
import numpy as np
import pyvips
from django.conf import settings
from mims.models import MIMSAlignment

def manipulate_image(image, angle, flip_hor):
    manipulated_image = pyvips.Image.new_from_array(image)
    if angle:
        manipulated_image = manipulated_image.rotate(angle, background=0)
    if flip_hor:
        manipulated_image = manipulated_image.fliphor()
    return manipulated_image.numpy()


def update_top_locations(top_locations, x, y, iou, angle, flip_hor):
    to_add = (x, y, iou, angle, flip_hor)
    # Check if this (x, y) should be added to top_locations
    if len(top_locations) < 3:
        top_locations.append(to_add)
        return top_locations

    min_iou = min([i[2] for i in top_locations])
    if iou < min_iou and len(top_locations) >= 3:
        return top_locations

    for i, (tx, ty, tiou, tangle, tflip_hor) in enumerate(top_locations):
        distance = np.sqrt((tx - x) ** 2 + (ty - y) ** 2)
        if distance < 40 and tflip_hor is flip_hor and abs(tangle - angle) < 30:
            if iou > tiou:
                top_locations[i] = to_add
                top_locations = sorted(top_locations, key=lambda x: x[2], reverse=True)[
                    :3
                ]
                return top_locations

    if iou > min_iou:
        top_locations.append(to_add)
        # Reorder by iou and remove the lowest
    top_locations = sorted(top_locations, key=lambda x: x[2], reverse=True)[:3]
    return top_locations


def do_sliding_search(im1, im2, valid_im1, valid_im2, angle, flip_hor):
    best_locations = []
    # Do a sliding search for im2 in im1
    for y in range(0, im1.shape[0] - im2.shape[0], 20):
        for x in range(0, im1.shape[1] - im2.shape[1], 20):
            im1_slice = im1[y : y + im2.shape[0], x : x + im2.shape[1]]
            if im1_slice.shape != im2.shape:
                print("this shouldnt happen", im1_slice.shape, im2.shape)
                continue
            valid_im1_slice = valid_im1[y : y + im2.shape[0], x : x + im2.shape[1]]
            valid_mask = valid_im1_slice & valid_im2
            intersection = np.logical_and(im1_slice, im2).sum()
            union = (
                np.logical_and(valid_mask, np.logical_or(im1_slice, im2))
                .astype(np.uint8)
                .sum()
            )
            if (
                valid_mask.sum() < 100000
                or union < 2000
            ):
                # Should have at least 100,000 valid pixels and 2000 union
                continue
            iou = 0
            if union:
                iou = intersection / union
                best_locations = update_top_locations(
                    best_locations, x, y, iou, angle, flip_hor
                )
    return best_locations


def non_zero_percentile(arr, percentile):
    """
    Calculate the percentile for all non-zero values in the given array.

    Parameters:
    arr (numpy.ndarray): Input array.
    percentile (float): Percentile to compute (0-100).

    Returns:
    float: The percentile value of the non-zero elements.
    """
    # Filter out the non-zero values
    non_zero_values = arr[arr != 0]

    # Calculate the percentile of the non-zero values
    percentile_value = np.percentile(non_zero_values, percentile)

    return percentile_value


def correct_inner_zeros(original_array):
    array = original_array.astype(np.uint8)
    # Correct the inner zeros to 1s
    # First set all 0s to 1s, then outer 1s to 0s
    # 1. Set all 0s to 1s
    array = np.where(array == 0, 1, array)
    # 2. Set outer 1s to 0s
    for y in range(0, array.shape[0] - 1):
        # left side
        for x in range(0, array.shape[1] - 1):
            value = array[y][x]
            if value not in [1]:
                break
            array[y][x] = 0
        # right side
        for x in range(array.shape[1] - 1, 0, -1):
            value = array[y][x]
            if value not in [1]:
                break
            array[y][x] = 0

    for x in range(0, array.shape[1] - 1):
        # top side
        for y in range(0, array.shape[0] - 1):
            value = array[y][x]
            if value not in [1]:
                break
            array[y][x] = 0
        # bottom side
        for y in range(array.shape[0] - 1, 0, -1):
            value = array[y][x]
            if value not in [1]:
                break
            array[y][x] = 0
    return array


In [41]:
def threshold_match(
    im1_full, im2_full, THRESHOLD_8BIT_BUFFER=10, angle=None, flip_hor=None, prefix=""
):
    valid_im1 = im1_full > 0
    valid_im2 = im2_full > 0
    im1_threshold = np.percentile(im1_full, 90) - THRESHOLD_8BIT_BUFFER
    im1 = np.where(im1_full > im1_threshold, 255, 0).astype(np.uint8)
    im2_threshold = np.percentile(im2_full, 90) - THRESHOLD_8BIT_BUFFER
    im2 = np.where(im2_full > im2_threshold, 255, 0).astype(np.uint8)

    best_overall_locations = []

    if angle is not None and flip_hor is not None:
        im2_manipulated = manipulate_image(im2, angle, flip_hor)
        best_overall_locations = [
            (i, (angle, flip_hor))
            for i in do_sliding_search(
                im1, im2_manipulated, valid_im1, valid_im2, angle, flip_hor
            )
        ]
    else:
        for test_angle in range(0, 360, 15):
            if test_angle in [0, 90, 180, 270]:
                print(
                    datetime.now(), "starting angle", test_angle, best_overall_locations
                )
            for test_flip_hor in [True, False]:
                im2_manipulated = manipulate_image(im2, test_angle, test_flip_hor)
                valid_im2 = im2_manipulated > 0
                im2_manipulated = np.where(
                    im2_manipulated > im2_threshold, 255, 0
                ).astype(np.uint8)
                best_for_iteration = do_sliding_search(
                    im1,
                    im2_manipulated,
                    valid_im1,
                    valid_im2,
                    test_angle,
                    test_flip_hor,
                )
                for top_for_iteration in best_for_iteration:
                    best_overall_locations = update_top_locations(
                        best_overall_locations,
                        top_for_iteration[0],
                        top_for_iteration[1],
                        top_for_iteration[2],
                        top_for_iteration[3],
                        top_for_iteration[4],
                    )
    for (index1, item) in enumerate(best_overall_locations):
        index = f"{prefix}_{index1}"
        mims = manipulate_image(im2_full, item[3], item[4])
        pyvips.Image.new_from_array(mims).write_to_file(
            f"/Users/chris/Downloads/emtest/{index}_mims.png"
        )
        mims_t = np.where(mims > im2_threshold, 255, 0).astype(np.uint8)
        pyvips.Image.new_from_array(mims_t).write_to_file(
            f"/Users/chris/Downloads/emtest/{index}_mims_t.png"
        )
        mims_valid = mims > 0
        pyvips.Image.new_from_array(mims_valid).write_to_file(
            f"/Users/chris/Downloads/emtest/{index}_mims_valid.png"
        )
        em = im1_full[item[1]:item[1]+mims.shape[1], item[0]:item[0]+mims.shape[0]]
        pyvips.Image.new_from_array(em).write_to_file(
            f"/Users/chris/Downloads/emtest/{index}_em.png"
        )
        em_t = im1[item[1]:item[1]+mims.shape[1], item[0]:item[0]+mims.shape[0]]
        pyvips.Image.new_from_array(em_t).write_to_file(
            f"/Users/chris/Downloads/emtest/{index}_em_t.png"
        )
        valid_em = em > 0
        pyvips.Image.new_from_array(valid_em).write_to_file(
            f"/Users/chris/Downloads/emtest/{index}_em_valid.png"
        )
        valid_mask = valid_em & mims_valid
        pyvips.Image.new_from_array(valid_mask).write_to_file(
            f"/Users/chris/Downloads/emtest/{index}_valid_both.png"
        )
        intersection = em_t & mims_t
        pyvips.Image.new_from_array(intersection).write_to_file(
            f"/Users/chris/Downloads/emtest/{index}_intersection.png"
        )
        union = (
            np.logical_and(valid_mask, np.logical_or(mims_t, em_t))
        )
        pyvips.Image.new_from_array(union).write_to_file(
            f"/Users/chris/Downloads/emtest/{index}_union.png"
        )
    return best_overall_locations

def create_alignment_estimates(mims_image, confirmed_alignment=None):
    print(
        f"{datetime.now()} starting alignment. Confirmed alignment is {confirmed_alignment}"
    )
    if confirmed_alignment:
        # Get the em coordinates of the mims_image given the confirmed one.
        print(
            f"Found confirmed alignment in project: angle of {confirmed_alignment.rotation_degrees}, flip_hor of {confirmed_alignment.flip_hor}"
        )
        confirmed_im_obj = sims.SIMS(confirmed_alignment.mims_image.file)
        current_im_obj = sims.SIMS(mims_image.mims_image.file)
        current_im_x = current_im_obj.header["sample x"]
        current_im_y = current_im_obj.header["sample y"]
        confirmed_im_obj_x = confirmed_im_obj.header["sample x"]
        confirmed_im_obj_y = confirmed_im_obj.header["sample y"]
        confirmed_alignment_em_x = confirmed_alignment.x_offset
        confirmed_alignment_em_y = confirmed_alignment.y_offset
        scale = confirmed_alignment.scale
        mims_pixel_size = mims_image.pixel_size_nm
        em_pixel_size = mims_image.image_set.em_image.pixel_size_nm
        angle = confirmed_alignment.rotation_degrees
        flip_hor = confirmed_alignment.flip_hor
        # Calculate the difference in microns between the current and confirmed nanoSIMS images
        delta_x_microns = current_im_x - confirmed_im_obj_x
        delta_y_microns = current_im_y - confirmed_im_obj_y

        # Convert the difference to EM pixels
        delta_x_em_pixels = delta_x_microns * 1000 / em_pixel_size
        delta_y_em_pixels = delta_y_microns * 1000 / em_pixel_size

        # Create rotation matrix
        theta = np.radians(angle)
        rotation_matrix = np.array(
            [[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]]
        )
        # Apply rotation to the delta coordinates
        rotated_delta = np.dot(
            rotation_matrix, np.array([delta_x_em_pixels, delta_y_em_pixels])
        )

        # Apply flip_hor if necessary
        if flip_hor:
            rotated_delta[0] = -rotated_delta[0]
        # Calculate the new EM coordinates
        new_em_x = confirmed_alignment_em_x + rotated_delta[0]
        new_em_y = confirmed_alignment_em_y + rotated_delta[1]

        # Apply scale factor
        new_em_x *= scale
        new_em_y *= scale

        # Round to nearest integer (assuming EM coordinates are in pixels)
        new_em_x = round(new_em_x)
        new_em_y = round(new_em_y)

        print(mims_image.file.filename)
        print(
            confirmed_im_obj_x,
            confirmed_im_obj_y,
            confirmed_alignment_em_x,
            confirmed_alignment_em_y,
        )
        print(new_em_x, new_em_y)
        mims_image.alignments.filter(status="FROM_SET").delete()
        new_from_set_alignment = MIMSAlignment(
            mims_image=mims_image,
            status="FROM_SET",
            x_offset=new_em_x,
            y_offset=new_em_y,
            rotation_degrees=confirmed_alignment.rotation_degrees,
            flip_hor=confirmed_alignment.flip_hor,
            scale=confirmed_alignment.scale,
        )
        new_from_set_alignment.save()
        return

    # Clear old alignments
    if hasattr(mims_image, "alignments") and mims_image.alignments.exists():
        mims_image.alignments.all().delete()

    # Get scale
    mims_pixel_size = mims_image.pixel_size_nm
    em_pixel_size = mims_image.image_set.em_image.pixel_size_nm
    scale = em_pixel_size / mims_pixel_size

    # Load the 32S image
    sulfur_image_path = os.path.join(
        settings.MEDIA_ROOT,
        "mims_images",
        str(mims_image.image_set.id),
        str(mims_image.id),
        "isotopes",
        "32S_autocontrast.png",
    )
    sulfur_image = pyvips.Image.new_from_file(sulfur_image_path).numpy()

    # Prep the 32S image:
    # Invert the 8-bit image
    sulfur_image = np.invert(sulfur_image)
    # Change 0 to 1 so we can ignore extra pixels from rotation
    sulfur_image = np.where(sulfur_image == 0, 1, sulfur_image)

    # Load and scale the em image
    em_image_path = mims_image.image_set.em_image.file.path
    em_image = pyvips.Image.new_from_file(em_image_path).resize(scale).numpy()

    # Set inner 0s to 1s so we can expand with more 0s
    em_image = correct_inner_zeros(em_image).astype(np.uint8)
    # Expand the image to include padding for the sliding check
    EM_PADDING = int(sulfur_image.shape[0] * 0.6)
    em_image = (
        pyvips.Image.new_from_array(em_image)
        .embed(
            EM_PADDING,
            EM_PADDING,
            em_image.shape[1] + 2 * EM_PADDING,
            em_image.shape[0] + 2 * EM_PADDING,
            background=0,
        )
        .numpy()
    )
    filename = mims_image.file.name.split('/')[-1].split('.')[0]
    print(datetime.now(), f"starting threshold match for {filename}")
    best_matches = threshold_match(em_image, sulfur_image, 10, None, None, filename)
    print(datetime.now(), best_matches)
    alignment_status = "ESTIMATE_INITIAL"
    for match in best_matches:
        # Correct for the padding and scaling
        actual_x = (match[0] - EM_PADDING) / scale
        actual_y = (match[1] - EM_PADDING) / scale
        print(
            "FINAL", EM_PADDING, match, actual_x, actual_y, match[2], match[3], match[4]
        )
        alignment = MIMSAlignment(
            mims_image=mims_image,
            x_offset=actual_x,
            y_offset=actual_y,
            rotation_degrees=match[3],
            flip_hor=match[4],
            scale=scale,
            status=alignment_status,
        )
        alignment.save()


In [42]:

# Fetch and print the test image
test_images = MIMSImage.objects.all()
for test in test_images:
    res = create_alignment_estimates(test)

2024-06-30 14:24:38.686228 starting alignment. Confirmed alignment is None
2024-06-30 14:24:44.362317 starting threshold match for 1035_position_1
2024-06-30 14:24:44.398658 starting angle 0 []
2024-06-30 14:25:49.292797 starting angle 90 []
2024-06-30 14:26:53.363939 starting angle 180 []
2024-06-30 14:27:57.890738 starting angle 270 []
2024-06-30 14:29:02.477884 []
2024-06-30 14:29:02.478100 starting alignment. Confirmed alignment is None
2024-06-30 14:29:08.140495 starting threshold match for 1035_position_2
2024-06-30 14:29:08.177301 starting angle 0 []
2024-06-30 14:30:12.519262 starting angle 90 [(1280, 1500, np.float64(0.5325918542569255), 0, False), (1380, 1500, np.float64(0.5256343198886795), 0, True), (1240, 1500, np.float64(0.5249731074026192), 0, False)]
2024-06-30 14:31:16.781060 starting angle 180 [(1360, 1420, np.float64(0.5640051577068043), 90, True), (1380, 1420, np.float64(0.5592045229121206), 90, True), (1360, 1440, np.float64(0.5536599880975996), 90, True)]
2024-06-

In [ ]:
for a in test_image.alignments.all():
    print(a.x_offset, a.y_offset, a.rotation_degrees, a.flip, a.scale)